In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re

In [2]:

data = pd.read_json('multinli_1.0/multinli_1.0_train.jsonl',lines=True)

In [6]:
data.head()

,annotator_labels,genre,gold_label,pairID,promptID,sentence1,sentence1_binary_parse,sentence1_parse,sentence2,sentence2_binary_parse,sentence2_parse
0,[neutral],government,neutral,31193n,31193,Conceptually cream skimming has two basic dime...,( ( Conceptually ( cream skimming ) ) ( ( has ...,(ROOT (S (NP (JJ Conceptually) (NN cream) (NN ...,Product and geography are what make cream skim...,( ( ( Product and ) geography ) ( ( are ( what...,(ROOT (S (NP (NN Product) (CC and) (NN geograp...
1,[entailment],telephone,entailment,101457e,101457,you know during the season and i guess at at y...,( you ( ( know ( during ( ( ( the season ) and...,(ROOT (S (NP (PRP you)) (VP (VBP know) (PP (IN...,You lose the things to the following level if ...,( You ( ( ( ( lose ( the things ) ) ( to ( the...,(ROOT (S (NP (PRP You)) (VP (VBP lose) (NP (DT...
2,[entailment],fiction,entailment,134793e,134793,One of our number will carry out your instruct...,( ( One ( of ( our number ) ) ) ( ( will ( ( (...,(ROOT (S (NP (NP (CD One)) (PP (IN of) (NP (PR...,A member of my team will execute your orders w...,( ( ( A member ) ( of ( my team ) ) ) ( ( will...,(ROOT (S (NP (NP (DT A) (NN member)) (PP (IN o...
3,[entailment],fiction,entailment,37397e,37397,How do you know? All this is their information...,( ( How ( ( ( do you ) know ) ? ) ) ( ( All th...,(ROOT (S (SBARQ (WHADVP (WRB How)) (SQ (VBP do...,This information belongs to them.,( ( This information ) ( ( belongs ( to them )...,(ROOT (S (NP (DT This) (NN information)) (VP (...
4,[neutral],telephone,neutral,50563n,50563,yeah i tell you what though if you go price so...,( yeah ( i ( ( tell you ) ( what ( ( though ( ...,(ROOT (S (VP (VB yeah) (S (NP (FW i)) (VP (VB ...,The tennis shoes have a range of prices.,( ( The ( tennis shoes ) ) ( ( have ( ( a rang...,(ROOT (S (NP (DT The) (NN tennis) (NNS shoes))...


In [7]:
data['gold_label'].value_counts()

contradiction    130903
neutral          130900
entailment       130899
Name: gold_label, dtype: int64

In [8]:
data["sentence1"]

0         Conceptually cream skimming has two basic dime...
1         you know during the season and i guess at at y...
2         One of our number will carry out your instruct...
3         How do you know? All this is their information...
4         yeah i tell you what though if you go price so...
5         my walkman broke so i'm upset now i just have ...
6         But a few Christian mosaics survive above the ...
7          (Read  for Slate 's take on Jackson's findings.)
8                                        Gays and lesbians.
9         At the end of Rue des Francs-Bourgeois is what...
10        I burst through a set of cabin doors, and fell...
11                             Fun for adults and children.
12        It's not that the questions they asked weren't...
13        Thebes held onto power until the 12th Dynasty,...
14        I don't mean to be glib about your concerns, b...
15                                Issues in Data Synthesis.
16                     well you see that

In [9]:
data["sentence2"]

0         Product and geography are what make cream skim...
1         You lose the things to the following level if ...
2         A member of my team will execute your orders w...
3                         This information belongs to them.
4                  The tennis shoes have a range of prices.
5         I'm upset that my walkman broke and now I have...
6         Most of the Christian mosaics were destroyed b...
7               Slate had an opinion on Jackson's findings.
8                                            Heterosexuals.
9         Place des Vosges is constructed entirely of gr...
10                 I burst through the doors and fell down.
11                                   Fun for only children.
12        All of the questions were interesting accordin...
13        The capital near Memphis lasted only half a ce...
14        I am concerned more about your issues than the...
15                              Problems in data synthesis.
16                 You can see that on t

In [3]:
glove_df = pd.read_csv('glove.6B/glove.6B.300d.txt', sep=" ", quoting=3, header=None, index_col=0)
glove = {key: val.values for key, val in glove_df.T.items()}

In [4]:
replace_puncts = {'`': "'", '′': "'", '“':'"', '”': '"', '‘': "'"}

strip_chars = [',', '.', '"', ':', ')', '(', '-', '|', ';', "'", '[', ']', '>', '=', '+', '\\', '•',  '~', '@', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

puncts = ['!', '?', '$', '&', '/', '%', '#', '*','£']

def clean_str(x):
    x = str(x)
    
    x = x.lower()
    
    x = re.sub(r"(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9]\.[^\s]{2,})", "url", x)
    
    for k, v in replace_puncts.items():
        x = x.replace(k, f' {v} ')
        
    for punct in strip_chars:
        x = x.replace(punct, ' ') 
    
    for punct in puncts:
        x = x.replace(punct, f' {punct} ')
        
    x = x.replace(" '", " ")
    x = x.replace("' ", " ")
        
    return x


In [5]:
data['clean_sentence1'] = data['sentence1'].apply(clean_str)

In [6]:
data['clean_sentence2'] = data['sentence2'].apply(clean_str)

In [14]:
! pip install keras 

In [7]:
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [8]:
X=[]
for i in range(len(data)):
    sent1_words = data["clean_sentence1"][i].split()
    sent2_words = data["clean_sentence2"][i].split()
    X.append(sent1_words+sent2_words)
    

In [9]:
data['sent1_sent2'] = [' '.join(map(str, l)) for l in X]

In [10]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
data["gold_label_encoded"] = le.fit_transform(data["gold_label"])

In [23]:
#y = data["gold_label_encoded"]

In [24]:
#data['sent1_sent2'][0],y[0]

('conceptually cream skimming has two basic dimensions product and geography product and geography are what make cream skimming work',
 2)

In [25]:
max_features = 20000 
sequence_length = 100

tokenizer = Tokenizer(num_words=max_features, split=' ', oov_token='<unw>', filters=' ')
tokenizer.fit_on_texts(data['sent1_sent2'].values)
X = tokenizer.texts_to_sequences(data['sent1_sent2'].values)

# we then pad the sequences so they're all the same length (sequence_length)
X = pad_sequences(X, sequence_length)


y = pd.get_dummies(data['gold_label']).values

# lets keep a couple of thousand samples back as a test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

print("test set size " + str(len(X_test)))

test set size 39271


In [12]:
max_features = 20000 
sequence_length = 100


tokenizer = Tokenizer(num_words=max_features, split=' ', oov_token='<unw>', filters=' ')
tokenizer.fit_on_texts(data['sent1_sent2'].values)
X_Hypothesis= tokenizer.texts_to_sequences(data['clean_sentence1'].values)

# we then pad the sequences so they're all the same length (sequence_length)
X_Hypothesis = pad_sequences(X_Hypothesis, sequence_length,padding = "post")

In [13]:
X_premise= tokenizer.texts_to_sequences(data['clean_sentence2'].values)

# we then pad the sequences so they're all the same length (sequence_length)
X_premise = pad_sequences(X_premise, sequence_length,padding = "post")

In [14]:
Input = []
for i in range(len(X_Hypothesis)):
    Input.append((X_Hypothesis[i],X_premise[i]))
    

In [15]:
Input = np.array(Input)

In [16]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
data["gold_label_encoded"] = le.fit_transform(data["gold_label"])

In [17]:
y = data["gold_label_encoded"]

In [18]:
'''required y - 0,1,2
to get in original label form:
do le.inverse_transform

'''
y[1]

1

In [21]:
data_for_training=[]
for i in range(len(X_Hypothesis)):
    data_for_training.append((Input[i],y[i]))

In [23]:
##### Data you need

data_for_training = np.array(data_for_training)

In [26]:
import os
embeddings_index = {}
f = open(os.path.join('glove.6B', 'glove.6B.300d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400001 word vectors.


In [27]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 70666 unique tokens.


In [28]:
num_words = min(max_features, len(word_index)) + 1
print(num_words)

embedding_dim = 300

# first create a matrix of zeros, this is our embedding matrix
embedding_matrix = np.zeros((num_words, embedding_dim))

# for each word in out tokenizer lets try to find that work in our w2v model
for word, i in word_index.items():
    if i > max_features:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # we found the word - add that words vector to the matrix
        embedding_matrix[i] = embedding_vector
    else:
        # doesn't exist, assign a random vector
        embedding_matrix[i] = np.random.randn(embedding_dim)

20001


In [29]:
embedding_matrix[1]

array([ 1.40321148e+00, -1.72426198e+00,  1.13416828e+00, -8.45881547e-01,
       -1.80816954e+00, -4.85328715e-02,  1.69591031e+00,  4.17371559e-01,
       -2.28151247e-01, -6.14572996e-01,  2.60956198e-01, -3.90093246e-01,
       -2.66945902e-02,  2.71881494e-01,  1.97920797e-01, -1.21738604e+00,
        2.02166286e+00, -1.45422490e+00, -2.93138143e+00,  1.49728355e+00,
       -1.19955804e+00, -1.46681275e+00,  1.18539437e+00, -1.63293255e+00,
        1.16645019e+00, -7.97348764e-01, -1.22255764e+00, -4.74676155e-02,
        4.06969275e-01, -1.41974262e-01,  1.03432055e+00,  7.70061024e-01,
        9.29975058e-01, -2.36867706e-01,  2.91366647e-01, -5.74054295e-01,
        1.57131245e+00, -1.02976898e+00, -2.52526656e-01,  1.73607447e+00,
        1.47474513e+00,  1.55667301e+00,  1.66977495e+00, -2.02412609e-01,
       -1.63701807e+00, -1.03146287e+00,  3.63289088e-01,  1.07893125e+00,
        1.39084959e+00,  1.74903851e-01,  9.29371221e-01, -2.23387312e-01,
        9.05979803e-01, -

In [30]:
from keras.layers import GRU,Embedding,Dense
from keras.models import Sequential

In [31]:
model = Sequential()
model.add(Embedding(num_words,
                    embedding_dim,
                    weights=[embedding_matrix],
                    input_length=sequence_length,
                    trainable=True))
model.add(GRU(256,activation="tanh",return_sequences=True))
model.add(GRU(128,activation="tanh"))
model.add(Dense(units=3, activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 300)          6000300   
_________________________________________________________________
gru_1 (GRU)                  (None, 100, 256)          427776    
_________________________________________________________________
gru_2 (GRU)                  (None, 128)               147840    
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 387       
Total params: 6,576,303
Trainable params: 6,576,303
Non-trainable params: 0
_________________________________________________________________
None


In [32]:
batch_size = 128
history = model.fit(X_train, y_train, epochs=5, batch_size=batch_size, verbose=1, validation_split=0.1)


Train on 318087 samples, validate on 35344 samples
Epoch 1/5
318087/318087 [==============================] - 3807s 12ms/step - loss: 0.9262 - acc: 0.5487 - val_loss: 0.8597 - val_acc: 0.5967
Epoch 2/5
318087/318087 [==============================] - 3013s 9ms/step - loss: 0.7993 - acc: 0.6379 - val_loss: 0.8304 - val_acc: 0.6188
Epoch 3/5
318087/318087 [==============================] - 2766s 9ms/step - loss: 0.6976 - acc: 0.6960 - val_loss: 0.8623 - val_acc: 0.6121
Epoch 4/5
318087/318087 [==============================] - 3452s 11ms/step - loss: 0.5822 - acc: 0.7558 - val_loss: 0.9459 - val_acc: 0.6017
Epoch 5/5
318087/318087 [==============================] - 3286s 10ms/step - loss: 0.4570 - acc: 0.8157 - val_loss: 1.0886 - val_acc: 0.5852


In [33]:
model.save('baseline_model.h5') 

from keras.models import load_model

In [34]:
model_baseline = load_model('baseline_model.h5')

In [35]:
predictions = model_baseline.predict(X_test)

In [42]:
pred_list = [np.argmax(predictions[i]) for i in range(len(predictions))] 

In [43]:
true_labels = [np.argmax(y_test[i]) for i in range(len(y_test))]

In [44]:
from sklearn.metrics import accuracy_score

In [46]:
accuracy_score(true_labels,pred_list)

0.5853174097934863

In [52]:
match =[]
mismatch=[]
for i in range(len(true_labels)):
    if true_labels[i]!=pred_list[i]:
        mismatch.append(i)
    else:
        match.append(i)

In [53]:
len(mismatch)

16285

In [72]:
len(match)

22986

In [74]:
mismatch[0]

0

In [57]:
X_test[0]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           2,  5733,   111,  1047,  3320,  4537,     4, 12114,     2,
        5733,    15,   756,  2068,    14,     2, 12114,     3,    22,
         475], dtype=int32)

In [59]:
reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))


In [67]:
text_X_test = tokenizer.sequences_to_texts(X_test)

In [70]:
mismatch_text=[]
for i in mismatch:
    mismatch_text.append(text_X_test[i])
    
    
    

In [76]:
mismatch_text = [mismatch_text[i].replace('<unw>',"") for i in range(len(mismatch_text))]
mismatch_text = 

In [77]:
mismatch_text[0]

'                                                                                 the pump also offers superb ease of squeeze the pump was designed specifically for the squeeze to be easy'

In [ ]:
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
fmri = sns.load_dataset("fmri")
ax = sns.lineplot(x="timepoint", y="signal", data=fmri)